In [ ]:
# default_exp play04_jpg
# pip3 install --user opencv-python opencv-contrib-python tqdm xarray pandas h5netcdf lmfit


In [ ]:
#export
import matplotlib
import matplotlib.pyplot as plt
plt.ion()
import time
import pathlib
import numpy as np
import cv2 as cv
import pandas as pd
import xarray as xr
import lmfit
import cv2.aruco
import tqdm
import decimal
from matplotlib.pyplot import plot, imshow, tight_layout, xlabel, ylabel, title, subplot, subplot2grid, grid, legend, figure, gcf, xlim, ylim


In [ ]:
#export
_code_git_version="f86745e09534ee036aa93c2afccf7f6f5ded2293"
_code_repository="https://github.com/plops/cl-py-generator/tree/master//example/76_opencv_cuda/source/04_load_jpg.ipynb"
_code_generation_time="18:17:41 of Wednesday, 2024-04-24 (GMT+1)"
start_time=time.time()
debug=True


In [ ]:
#export
df_status=pd.DataFrame([dict(name="numpy", version=np.__version__), dict(name="cv2", version=cv.__version__), dict(name="pandas", version=pd.__version__), dict(name="xarray", version=xr.__version__), dict(name="lmfit", version=lmfit.__version__)])
print(df_status)


In [ ]:
#export
# https://answers.opencv.org/question/98447/camera-calibration-using-charuco-and-python/
aruco_dict=cv.aruco.getPredefinedDictionary(cv.aruco.DICT_4X4_1000)
squares_x=49
squares_y=28
square_length=2
marker_length=1
board=cv.aruco.CharucoBoard_create(squares_x, squares_y, square_length, marker_length, aruco_dict)
out_size=(1960,1120,)
board_img=board.draw(out_size)
steps_x=5
steps_y=5
print("aruco dictionary can correct at most {} bits".format(aruco_dict.maxCorrectionBits))


In [ ]:
#export
xs_fn="calib03/checkerboards.nc"
if ( pathlib.Path(xs_fn).exists() ):
    start=time.time()
    xs=xr.open_dataset(xs_fn)
    print("duration loading from netcdf {:4.2f}s".format(((time.time())-(start))))
else:
    start=time.time()
    fns=list(pathlib.Path("calib03/").glob("*.jpg"))
    res=[]
    for fn in tqdm.tqdm(fns):
        rgb=cv.imread(str(fn))
        gray=cv.cvtColor(rgb, cv.COLOR_BGR2GRAY)
        res.append(gray)
    data=np.stack(res, 0)
    xs=xr.Dataset(dict(cb=xr.DataArray(data=data, dims=["frame", "h", "w"], coords=dict(frame=np.arange(data.shape[0]), h=np.arange(data.shape[1]), w=np.arange(data.shape[2])))))
    xs.to_netcdf(xs_fn)
    print("duration loading from jpg and saving netcdf {:4.2f}s".format(((time.time())-(start))))


In [ ]:
#export
# this will be used by interpolateCornersCharuco
# initially with out camera matrix ApproxCalib will be run
# you can execute the next cell again after camera_matrix has been found to run LocalHom
camera_matrix=None
distortion_params=None


In [ ]:
#export
# flags for camera calibration that influence the model (fix fx=fy, number of coefficents for distortion)
# fix K1,K2 and K3 to zero (no distortion)
calibrate_camera_flags_general=((cv.CALIB_ZERO_TANGENT_DIST) | (cv.CALIB_FIX_ASPECT_RATIO) | (cv.CALIB_FIX_K1) | (cv.CALIB_FIX_K2) | (cv.CALIB_FIX_K3))


In [ ]:
# use this for the second run to make use of existing camera matrix
calibrate_camera_flags=((cv.CALIB_USE_INTRINSIC_GUESS) | (calibrate_camera_flags_general))


In [ ]:
#export
# use these flags for the first run
calibrate_camera_flags=((calibrate_camera_flags_general))


In [ ]:
#export
# collect corners for each frame
do_plot=False
save_figure=False
w="cb"
if ( do_plot ):
    cv.namedWindow(w, cv.WINDOW_NORMAL)
    cv.resizeWindow(w, 1600, 900)
all_corners=[]
all_ids=[]
all_rejects=[]
aruco_params=cv.aruco.DetectorParameters_create()
for frame in tqdm.tqdm(range(len(xs.frame))):
    gray=xs.cb[frame,...].values
    # rejected_points[NR-1].shape = 1 4 2, NR=566
    corners, ids, rejected_points=cv.aruco.detectMarkers(image=gray, dictionary=aruco_dict, parameters=aruco_params, cameraMatrix=camera_matrix, distCoeff=distortion_params)
    if ( ((0)<(len(corners))) ):
        # corners[N-1].shape = 1 4 2, for each of N markers provide 4 corners
# ids[0] = 653, id for this particular marker
# cameraMatrix (optional) [fx 0 cx; 0 fy c0; 0 0 1]
# distCoeffs (optional 4,5,8 or 12 elements) k1 k2 p1 p1 [k3 [k4 k5 k6] [s1 s2 s3 s4]]
# minMarkers (optional) number of adjacent markers that must be detected to return corner
        charuco_retval, int_corners, int_ids=cv.aruco.interpolateCornersCharuco(markerCorners=corners, markerIds=ids, image=gray, board=board, cameraMatrix=camera_matrix)
        if ( ((20)<(charuco_retval)) ):
            # found at least 20 squares
            all_corners.append(int_corners)
            all_ids.append(int_ids)
            all_rejects.append(rejected_points)
        if ( save_figure ):
            # image 16 and 25 have the most recognized markers (i think)
# blue .. markers, index fixed to board, starts from top left, increases towards right
# green .. corners, fixed to board, starts from bottom left increases towards right
            img=cv.aruco.drawDetectedCornersCharuco(image=cv.cvtColor(gray, cv.COLOR_GRAY2RGB), charucoCorners=int_corners, charucoIds=int_ids, cornerColor=(255,255,0,))
            cv.aruco.drawDetectedMarkers(img, corners, ids, (0,255,0,))
            cv.imwrite("/dev/shm/{:02d}.jpg".format(frame), img)
        if ( do_plot ):
            cv.imshow(w, gray[::4,::4])
            cv.setWindowTitle(w, "frame {}".format(frame))
            cv.waitKey(1)
# all_corners[0].shape = 295 1 2
# all_ids[0].shape     = 295 1
try:
    calibration, camera_matrix, distortion_params, rvecs, tvecs=cv.aruco.calibrateCameraCharuco(charucoCorners=all_corners, charucoIds=all_ids, board=board, imageSize=gray.shape, cameraMatrix=camera_matrix, distCoeffs=distortion_params, flags=calibrate_camera_flags)
except Exception as e:
    print(e)
    pass
print(camera_matrix)
print(distortion_params)
# board.chessboardCorners.shape 1296 3
# all marker corners on the board
# board.objPoints[685].shape 4 3;  coordinates of 4 points in CCW order,  z coordinate 0
if ( do_plot ):
    cv.destroyAllWindows()


In [ ]:
# calibration step by itself, also print fit errors for the parameters
# intrinsics: fx,fy,cx,cy,k1,k2,p1,p2,k3,k4,k5,k6,s1,s2,s3,s4,τx,τy
# extrinsics: R0,T0,…,RM−1,TM−1
# M .. number of frames
# R_i, T_i .. concatenated 1x3 vectors
try:
    calibration3, camera_matrix3, distortion_params3, rvecs3, tvecs3, intrinsic_err, extrinsic_err, view_err=cv.aruco.calibrateCameraCharucoExtended(charucoCorners=all_corners, charucoIds=all_ids, board=board, imageSize=gray.shape, cameraMatrix=camera_matrix, distCoeffs=distortion_params, flags=calibrate_camera_flags)
except Exception as e:
    print(e)
    pass
print(camera_matrix3)
print(distortion_params3)
M=camera_matrix3
D=distortion_params3
for idx, (name,val,) in enumerate(zip(["fx", "fy", "cx", "cy", "k1", "k2", "p1", "p2", "k3"], [M[0,0], M[1,1], M[0,2], M[1,2], D[0,0], D[0,1], D[0,2], D[0,3], D[0,4]])):
    print("{} = {}±{} ({:2.1f}%)".format(name, decimal.Decimal("{:.4g}".format(val)).normalize().to_eng_string(), decimal.Decimal("{:.1g}".format(intrinsic_err[idx].item())).normalize().to_eng_string(), np.abs(((100)*(((intrinsic_err[idx].item())/(val)))))))


In [ ]:
# collect the data, so that i can implement the fit myself
# function calibrateCameraCharuco https://github.com/opencv/opencv_contrib/blob/a26f71313009c93d105151094436eecd4a0990ed/modules/aruco/src/charuco.cpp
assert(((0)<(len(all_ids))))
assert(((len(all_ids))==(len(all_corners))))
res=[]
for i, ids in enumerate(all_ids):
    n_corners=len(ids)
    corners=all_corners[i]
    assert(((0)<(n_corners)))
    assert(((n_corners)==(len(corners))))
    for j in range(n_corners):
        point_id=ids[j]
        assert(((0)<=(point_id)))
        assert(((point_id)<(len(board.chessboardCorners))))
        res.append(dict(frame_idx=i, corner_idx=j, point_id=point_id.item(), x=board.chessboardCorners[point_id][0,0], y=board.chessboardCorners[point_id][0,1], u=corners[j,0,0], v=corners[j,0,1]))
df=pd.DataFrame(res)
df


In [ ]:
# plot the coordinates
plt.scatter(df.x, df.y)
plt.xlim(0, ((2)*(squares_x)))
plt.ylim(0, ((2)*(squares_y)))
grid()


In [ ]:
# heatmap of point density/coverage of the camera
fac=3
plt.hist2d(df.u, df.v, bins=[np.linspace(0, ((-1)+(xs.w.max().item())), ((16)*(fac))), np.linspace(0, ((-1)+(xs.h.max().item())), ((9)*(fac)))], cmap="cubehelix")
plt.colorbar()
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
grid()


In [ ]:
# try the transform with a single corner
frame_idx=24
rvec=rvecs[frame_idx]
tvec=tvecs[frame_idx]
R3, R3jac=cv.Rodrigues(src=rvec)
W=np.hstack([R3[:,0:2], tvec])
Mcam=camera_matrix3
D=distortion_params3
fx=Mcam[0,0]
fy=Mcam[1,1]
cx=Mcam[0,2]
cy=Mcam[1,2]
k1=D[0,0]
k2=D[0,1]
p1=D[0,2]
p2=D[0,3]
k3=D[0,4]
# https://docs.opencv.org/4.5.5/d9/d0c/group__calib3d.html#ga3207604e4b1a1758aa66acb6ed5aa65d Detailed description explains how to compute r
# documentation of undistortPoints explains how modify coordinates: https://docs.opencv.org/3.4/da/d54/group__imgproc__transform.html#ga55c716492470bfe86b0ee9bf3a1f0f7e 
# maybe this is clearer: https://docs.opencv.org/4.5.5/d9/d0c/group__calib3d.html#ga7dfb72c9cf9780a347fbe3d1c47e5d5a initUndistortRectifyMap
# here they don't have to perform undistort of the camera points but distortion of the model's coordinates
# fx_prime and cx_prime are parameters from the new camera matrix
M=np.array([[fx, 0, cx], [0, fx, cy], [0, 0, 1]])
df_=df[((df.frame_idx)==(frame_idx))]
row=df_.iloc[120]
res=[]
for idx, row in df_.iterrows():
    Q=np.array([[row.x], [row.y], [1]])
    WQ=np.matmul(W, Q)
    MWQ=np.matmul(M, WQ)
    # divide by 3rd coordinate, this will return x_prime y_prime
    mwq=((MWQ)/(MWQ[2]))
    # for each point in the corrected image compute the corresponding point in the (distorted) camera image 
    uv=np.array([[row.u], [row.v]])
    center=np.array([[cx], [cy]])
    F=np.array([[fx], [fy]])
    xy_=((((uv)-(center)))/(F))
    r2=((((xy_[0])**(2)))+(((xy_[1])**(2))))
    mwq_=((xy_)*(((1)+(((k1)*(r2))))))
    mwq_distorted=((((mwq_)*(F)))+(center))
    uv_pinhole_=cv.undistortPoints(src=uv, cameraMatrix=camera_matrix, distCoeffs=distortion_params).reshape([2, 1])
    uv_pinhole=((((uv_pinhole_)*(F)))+(center))
    # project checkerboard object into image
# https://docs.opencv.org/4.x/d4/d94/tutorial_camera_calibration.html computeReprojectionErrors
    uv_proj, uv_proj_jac=cv.projectPoints(objectPoints=Q, rvec=rvec, tvec=tvec, cameraMatrix=camera_matrix, distCoeffs=distortion_params)
    res.append(dict(mwq=mwq, mwq_distorted=mwq_distorted, uv=uv, uv_pinhole=uv_pinhole, uv_proj=uv_proj))
dft=pd.DataFrame(res)
dft


In [ ]:
plt.scatter(np.stack(dft.uv.values).squeeze()[:,0], np.stack(dft.uv.values).squeeze()[:,1], s=1)
grid()
xlabel("u")
ylabel("v")


In [ ]:
plt.scatter(np.stack(dft.uv_proj.values).squeeze()[:,0], np.stack(dft.uv_proj.values).squeeze()[:,1], s=1)
grid()
xlabel("u")
ylabel("v")


In [ ]:
plt.scatter(np.stack(dft.mwq.values).squeeze()[:,0], np.stack(dft.mwq.values).squeeze()[:,1], s=1)
grid()
xlabel("x_prime")
ylabel("y_prime")


In [ ]:
plt.scatter(np.stack(dft.uv_pinhole.values).squeeze()[:,0], np.stack(dft.uv_pinhole.values).squeeze()[:,1], s=1)
grid()
xlabel("x_pprime")
ylabel("y_pprime")


In [ ]:
plt.scatter(np.stack(dft.mwq_distorted.values).squeeze()[:,0], np.stack(dft.mwq_distorted.values).squeeze()[:,1], s=1)
grid()
xlabel("u")
ylabel("v")


In [ ]:
figure(figsize=[16, 9])
plt.scatter(np.stack(dft.uv.values).squeeze()[:,0], np.stack(dft.uv.values).squeeze()[:,1], marker="+")
plt.scatter(np.stack(dft.mwq.values).squeeze()[:,0], np.stack(dft.mwq.values).squeeze()[:,1], marker="x")
grid()
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
xlabel("x")
ylabel("y")


In [ ]:
figure(figsize=[16, 9])
plt.scatter(np.stack(dft.uv.values).squeeze()[:,0], np.stack(dft.uv.values).squeeze()[:,1], marker="+")
plt.scatter(np.stack(dft.mwq_distorted.values).squeeze()[:,0], np.stack(dft.mwq_distorted.values).squeeze()[:,1], marker="x")
grid()
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
xlabel("x")
ylabel("y")


In [ ]:
figure(figsize=[16, 9])
plt.scatter(np.stack(dft.uv.values).squeeze()[:,0], np.stack(dft.uv.values).squeeze()[:,1], marker="+")
plt.scatter(np.stack(dft.uv_pinhole.values).squeeze()[:,0], np.stack(dft.uv_pinhole.values).squeeze()[:,1], marker="x")
grid()
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
xlabel("x")
ylabel("y")


In [ ]:
figure(figsize=[16, 9])
plt.scatter(np.stack(dft.uv.values).squeeze()[:,0], np.stack(dft.uv.values).squeeze()[:,1], marker="+")
plt.scatter(np.stack(dft.uv_proj.values).squeeze()[:,0], np.stack(dft.uv_proj.values).squeeze()[:,1], marker="x")
grid()
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
xlabel("x")
ylabel("y")


In [ ]:
figure(figsize=[16, 9])
plt.scatter(np.stack(dft.uv_pinhole.values).squeeze()[:,0], np.stack(dft.uv_pinhole.values).squeeze()[:,1], marker="+")
plt.scatter(np.stack(dft.mwq.values).squeeze()[:,0], np.stack(dft.mwq.values).squeeze()[:,1], marker="x")
grid()
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
xlabel("x")
ylabel("y")


In [ ]:
figure(figsize=[16, 9])
# quiver plot to show mismatch between camera coordinates and transformed object coordinates
x0=np.stack(dft.uv.values).squeeze()[:,0]
y0=np.stack(dft.uv.values).squeeze()[:,1]
x1=np.stack(dft.mwq.values).squeeze()[:,0]
y1=np.stack(dft.mwq.values).squeeze()[:,1]
dx=((x0)-(x1))
dy=((y0)-(y1))
mi=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), 2)
ma=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), ((100)-(2)))
mi1=((mi)-((((0.10    ))*(((ma)-(mi))))))
ma1=((ma)+((((0.10    ))*(((ma)-(mi))))))
s=20
plt.quiver(x0, y0, dx, dy, scale=((s)*(ma1)))
plt.scatter([cx], [cy], marker="x", color="r")
grid()
title("compare uv and mwq max={:6.4f}".format(ma1))
plt.axis("equal")
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
xlabel("x")
ylabel("y")


In [ ]:
figure(figsize=[16, 9])
# quiver plot to show mismatch between camera coordinates and transformed object coordinates
x0=np.stack(dft.uv.values).squeeze()[:,0]
y0=np.stack(dft.uv.values).squeeze()[:,1]
x1=np.stack(dft.mwq_distorted.values).squeeze()[:,0]
y1=np.stack(dft.mwq_distorted.values).squeeze()[:,1]
dx=((x0)-(x1))
dy=((y0)-(y1))
mi=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), 2)
ma=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), ((100)-(2)))
mi1=((mi)-((((0.10    ))*(((ma)-(mi))))))
ma1=((ma)+((((0.10    ))*(((ma)-(mi))))))
s=20
plt.quiver(x0, y0, dx, dy, scale=((s)*(ma1)))
plt.scatter([cx], [cy], marker="x", color="r")
grid()
title("compare uv and mwq_distorted max={:6.4f}".format(ma1))
plt.axis("equal")
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
xlabel("x")
ylabel("y")


In [ ]:
figure(figsize=[16, 9])
# quiver plot to show mismatch between camera coordinates and transformed object coordinates
x0=np.stack(dft.uv.values).squeeze()[:,0]
y0=np.stack(dft.uv.values).squeeze()[:,1]
x1=np.stack(dft.uv_proj.values).squeeze()[:,0]
y1=np.stack(dft.uv_proj.values).squeeze()[:,1]
dx=((x0)-(x1))
dy=((y0)-(y1))
mi=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), 2)
ma=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), ((100)-(2)))
mi1=((mi)-((((0.10    ))*(((ma)-(mi))))))
ma1=((ma)+((((0.10    ))*(((ma)-(mi))))))
s=20
plt.quiver(x0, y0, dx, dy, scale=((s)*(ma1)))
plt.scatter([cx], [cy], marker="x", color="r")
grid()
title("compare uv and uv_proj max={:6.4f}".format(ma1))
plt.axis("equal")
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
xlabel("x")
ylabel("y")


In [ ]:
figure(figsize=[16, 9])
# quiver plot to show mismatch between camera coordinates and transformed object coordinates
x0=np.stack(dft.mwq.values).squeeze()[:,0]
y0=np.stack(dft.mwq.values).squeeze()[:,1]
x1=np.stack(dft.uv_proj.values).squeeze()[:,0]
y1=np.stack(dft.uv_proj.values).squeeze()[:,1]
dx=((x0)-(x1))
dy=((y0)-(y1))
mi=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), 2)
ma=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), ((100)-(2)))
mi1=((mi)-((((0.10    ))*(((ma)-(mi))))))
ma1=((ma)+((((0.10    ))*(((ma)-(mi))))))
s=20
plt.quiver(x0, y0, dx, dy, scale=((s)*(ma1)))
plt.scatter([cx], [cy], marker="x", color="r")
grid()
title("compare mwq and uv_proj max={:6.4f}".format(ma1))
plt.axis("equal")
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
xlabel("x")
ylabel("y")


In [ ]:
figure(figsize=[16, 9])
# quiver plot to show mismatch between camera coordinates and transformed object coordinates
x0=np.stack(dft.mwq_distorted.values).squeeze()[:,0]
y0=np.stack(dft.mwq_distorted.values).squeeze()[:,1]
x1=np.stack(dft.uv_proj.values).squeeze()[:,0]
y1=np.stack(dft.uv_proj.values).squeeze()[:,1]
dx=((x0)-(x1))
dy=((y0)-(y1))
mi=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), 2)
ma=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), ((100)-(2)))
mi1=((mi)-((((0.10    ))*(((ma)-(mi))))))
ma1=((ma)+((((0.10    ))*(((ma)-(mi))))))
s=20
plt.quiver(x0, y0, dx, dy, scale=((s)*(ma1)))
plt.scatter([cx], [cy], marker="x", color="r")
grid()
title("compare mwq_distorted and uv_proj max={:6.4f}".format(ma1))
plt.axis("equal")
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
xlabel("x")
ylabel("y")


In [ ]:
figure(figsize=[16, 9])
# quiver plot to show mismatch between camera coordinates and transformed object coordinates
x0=np.stack(dft.uv_pinhole.values).squeeze()[:,0]
y0=np.stack(dft.uv_pinhole.values).squeeze()[:,1]
x1=np.stack(dft.uv_proj.values).squeeze()[:,0]
y1=np.stack(dft.uv_proj.values).squeeze()[:,1]
dx=((x0)-(x1))
dy=((y0)-(y1))
mi=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), 2)
ma=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), ((100)-(2)))
mi1=((mi)-((((0.10    ))*(((ma)-(mi))))))
ma1=((ma)+((((0.10    ))*(((ma)-(mi))))))
s=20
plt.quiver(x0, y0, dx, dy, scale=((s)*(ma1)))
plt.scatter([cx], [cy], marker="x", color="r")
grid()
title("compare uv_pinhole and uv_proj max={:6.4f}".format(ma1))
plt.axis("equal")
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
xlabel("x")
ylabel("y")


In [ ]:
figure(figsize=[16, 9])
# quiver plot to show mismatch between camera coordinates and transformed object coordinates
x0=np.stack(dft.uv.values).squeeze()[:,0]
y0=np.stack(dft.uv.values).squeeze()[:,1]
x1=np.stack(dft.uv_proj.values).squeeze()[:,0]
y1=np.stack(dft.uv_proj.values).squeeze()[:,1]
dx=((x0)-(x1))
dy=((y0)-(y1))
mi=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), 2)
ma=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), ((100)-(2)))
mi1=((mi)-((((0.10    ))*(((ma)-(mi))))))
ma1=((ma)+((((0.10    ))*(((ma)-(mi))))))
s=20
plt.quiver(x0, y0, dx, dy, scale=((s)*(ma1)))
plt.scatter([cx], [cy], marker="x", color="r")
grid()
title("compare uv and uv_proj max={:6.4f}".format(ma1))
plt.axis("equal")
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
xlabel("x")
ylabel("y")


In [ ]:
figure(figsize=[16, 9])
# quiver plot to show mismatch between camera coordinates and transformed object coordinates
x0=np.stack(dft.mwq.values).squeeze()[:,0]
y0=np.stack(dft.mwq.values).squeeze()[:,1]
x1=np.stack(dft.uv_pinhole.values).squeeze()[:,0]
y1=np.stack(dft.uv_pinhole.values).squeeze()[:,1]
dx=((x0)-(x1))
dy=((y0)-(y1))
mi=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), 2)
ma=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), ((100)-(2)))
mi1=((mi)-((((0.10    ))*(((ma)-(mi))))))
ma1=((ma)+((((0.10    ))*(((ma)-(mi))))))
s=20
plt.quiver(x0, y0, dx, dy, scale=((s)*(ma1)))
plt.scatter([cx], [cy], marker="x", color="r")
grid()
title("compare mwq and uv_pinhole max={:6.4f}".format(ma1))
plt.axis("equal")
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
xlabel("x")
ylabel("y")


In [ ]:
figure(figsize=[16, 9])
# quiver plot to show mismatch between camera coordinates and transformed object coordinates
x0=np.stack(dft.uv.values).squeeze()[:,0]
y0=np.stack(dft.uv.values).squeeze()[:,1]
x1=np.stack(dft.uv_pinhole.values).squeeze()[:,0]
y1=np.stack(dft.uv_pinhole.values).squeeze()[:,1]
dx=((x0)-(x1))
dy=((y0)-(y1))
mi=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), 2)
ma=np.nanpercentile(np.sqrt(((((dx)**(2)))+(((dy)**(2))))), ((100)-(2)))
mi1=((mi)-((((0.10    ))*(((ma)-(mi))))))
ma1=((ma)+((((0.10    ))*(((ma)-(mi))))))
s=20
plt.quiver(x0, y0, dx, dy, scale=((s)*(ma1)))
plt.scatter([cx], [cy], marker="x", color="r")
grid()
title("compare uv and uv_pinhole max={:6.4f}".format(ma1))
plt.axis("equal")
plt.xlim(0, ((-1)+(xs.w.max().item())))
plt.ylim(0, ((-1)+(xs.h.max().item())))
xlabel("x")
ylabel("y")


In [ ]:
# show distortion for the corners in this particular frame
r=np.sqrt(((((((x0)-(cx)))**(2)))+(((((y0)-(cy)))**(2)))))
plt.scatter(r, np.sqrt(((((dx)**(2)))+(((dy)**(2))))))
xlim(0, 2500)
grid()
xlabel("r")
ylabel("dr")


In [ ]:
# distortion should be monotonic (barrel distortion decreases). if not then consider calibration a failure
r=np.linspace(0, 1)
plot(r, ((1)+(((k1)*(((r)**(2)))))+(((k2)*(((r)**(4)))))+(((k3)*(((r)**(6)))))))
grid()
xlabel("r")
ylabel("distortion factor")
